In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#movies = pd.read_csv('https://raw.githubusercontent.com/masumhasan/Movie-Recommender-AI/main/tmdb_5000_movies.csv')
#credits = pd.read_csv('https://raw.githubusercontent.com/masumhasan/Movie-Recommender-AI/main/tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(2)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
credits.head(1)['crew'].values
# show 1 crew values

Marge 2 dataset

In [ ]:
#daframe marge code

#movies.merge(credits,on='title')
#check size
# movies.merge(credits,on='title').shape
#assign new df in movies
movies = movies.merge(credits,on='title')

In [ ]:
#check size
movies.shape
#credits.shape

In [ ]:
#df info
movies.info()

In [ ]:
#movies_id
#title
#genres
#keywords
#overview
#cast
#crew

In [ ]:
#storing only prefered columns back into movies df
movies = movies[['movie_id','title','genres','overview','keywords','cast','crew']]

In [ ]:
#df info
movies.info()

In [ ]:
movies.head(1)

Data Preprocessing


In [ ]:
#check for missing data
movies.isnull().sum()

In [ ]:
#Drop missing datas
movies.dropna(inplace=True)

In [ ]:
#Check for duplicate data
movies.duplicated().sum()

In [ ]:
#Check data
movies.iloc[0].genres

Preprocessing



In [ ]:
#GOAL
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Adventure','Fantacy']

In [ ]:
#Convert string of list to list 
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
#helper function
def converttoL(obj):
    L= []
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        L.append(i['name'])
    return L    

In [ ]:
#converting genres to a list from string
movies['genres'] = movies['genres'].apply(converttoL) 

In [ ]:
movies.head(1)

In [ ]:
#converting keywords to a list from string
movies['keywords'] = movies['keywords'].apply(converttoL) 

In [ ]:
movies.head(1)

In [ ]:
#helper function
def convertcast(obj):
    L= []
    counter = 0
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        if counter !=4: #looping for 4 cast only
            L.append(i['name'])
            counter +=1
        else:
             break
    return L    

In [ ]:
#converting casts to a list from string
movies['cast'] = movies['cast'].apply(convertcast) 

In [ ]:
movies['cast'][0]

In [ ]:
movies.head()

In [ ]:
#Helper function
def fDirector(obj):
    L= []
    for i in ast.literal_eval(obj): 
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L    

In [ ]:
#converting casts (Director) to a list from string
movies['crew'] = movies['crew'].apply(fDirector)

In [ ]:
movies.head(1)

overview

In [ ]:
movies['overview'][0]
#its a string

In [ ]:
# spliting the overview data into words with lambda split function
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)